In [ ]:
# {echo=FALSE}
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd

project = 'covid-19-task' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

# Metrics
Metrics are often citations and publication numbers. I chose a weighted metric based on these distribution stats that balanced fairness to field variation in output and citation practices while preserving the impact from field citation ratio.

In [ ]:
# {echo=FALSE}

from google.cloud import bigquery
import pandas as pd
import numpy as np

# Initialize the BigQuery client
client = bigquery.Client()

# 1. Citation Rate (Total Citations / Total Publications): Global Descriptive Statistics
query_citations = """
SELECT
    research_org_country_names[SAFE_OFFSET(0)] AS country,
    SUM(citations_count) AS total_citations,
    COUNT(DISTINCT id) AS total_publications
FROM
    `covid-19-dimensions-ai.data.publications`
WHERE
    ARRAY_LENGTH(research_org_country_names) > 0
    AND citations_count IS NOT NULL
GROUP BY
    country
"""
df_citations = client.query(query_citations).to_dataframe()
df_citations['citations_per_publication'] = df_citations['total_citations'] / df_citations['total_publications']
citations_stats = df_citations['citations_per_publication'].describe().to_dict()

# 2. Research Impact (Average FCR): Grouped by Field, then Global Descriptive Statistics
fcr_query = """
SELECT
    category_for.first_level.full[SAFE_OFFSET(0)].name AS field_name,
    AVG(metrics.field_citation_ratio) AS avg_fcr
FROM
    `covid-19-dimensions-ai.data.publications`
WHERE metrics.field_citation_ratio IS NOT NULL
GROUP BY field_name
"""
fcr_df = client.query(fcr_query).to_dataframe()
fcr_stats = fcr_df['avg_fcr'].describe().to_dict()

# 3. Research Efficiency (Average FCR / Total Publications): Grouped by Field, then Country, then Global Descriptive Statistics
query_efficiency = """
SELECT
    research_org_country_names[SAFE_OFFSET(0)] AS country,
    category_for.first_level.full[SAFE_OFFSET(0)].name AS field_name,
    AVG(metrics.field_citation_ratio) AS avg_fcr,
    COUNT(DISTINCT id) AS publication_count
FROM
    `covid-19-dimensions-ai.data.publications`
WHERE metrics.field_citation_ratio IS NOT NULL
GROUP BY country, field_name
"""
df_efficiency = client.query(query_efficiency).to_dataframe()
df_efficiency['research_efficiency'] = df_efficiency['avg_fcr'] / df_efficiency['publication_count']
efficiency_stats = df_efficiency['research_efficiency'].describe().to_dict()

# 4. Impact-Weighted Research Output: Modify to Use Adjusted FCR
query_weighted = """
SELECT
    category_for.first_level.full[SAFE_OFFSET(0)].name AS field_name,
    metrics.field_citation_ratio AS fcr,
    COUNT(DISTINCT id) AS publication_count
FROM
    `covid-19-dimensions-ai.data.publications`
WHERE metrics.field_citation_ratio IS NOT NULL
GROUP BY field_name, fcr
"""
df_weighted = client.query(query_weighted).to_dataframe()
field_avg_fcr = df_weighted.groupby('field_name')['fcr'].transform('mean')
df_weighted['normalized_fcr'] = df_weighted['fcr'] / field_avg_fcr
df_weighted['log_publication_count'] = np.log1p(df_weighted['publication_count'])
df_weighted['weighted_fcr'] = df_weighted['normalized_fcr'] * df_weighted['log_publication_count']
df_weighted['normalized_weighted_fcr'] = df_weighted['weighted_fcr'] / df_weighted['weighted_fcr'].max()
weighted_stats = df_weighted['normalized_weighted_fcr'].describe().to_dict()

# Create a DataFrame for export
data = {
    "Metric": ["Citation Rate", "Average FCR", "Research Efficiency", "Impact-Weighted Research Output"],
    "Count": [citations_stats['count'], fcr_stats['count'], efficiency_stats['count'], weighted_stats['count']],
    "Mean": [citations_stats['mean'], fcr_stats['mean'], efficiency_stats['mean'], weighted_stats['mean']],
    "Std Dev": [citations_stats['std'], fcr_stats['std'], efficiency_stats['std'], weighted_stats['std']],
    "Min": [citations_stats['min'], fcr_stats['min'], efficiency_stats['min'], weighted_stats['min']],
    "25%": [citations_stats['25%'], fcr_stats['25%'], efficiency_stats['25%'], weighted_stats['25%']],
    "Median": [citations_stats['50%'], fcr_stats['50%'], efficiency_stats['50%'], weighted_stats['50%']],
    "75%": [citations_stats['75%'], fcr_stats['75%'], efficiency_stats['75%'], weighted_stats['75%']],
    "Max": [citations_stats['max'], fcr_stats['max'], efficiency_stats['max'], weighted_stats['max']],
    "Variance": [citations_stats['std']**2, fcr_stats['std']**2, efficiency_stats['std']**2, weighted_stats['std']**2],
}

final_df = pd.DataFrame(data)
print(final_df.to_string(index=False))

# save to gbq table for export
import pandas_gbq
pandas_gbq.to_gbq(final_df, "final_data.first_table_metrics", project_id="covid-19-task")

                         Metric   Count      Mean  Std Dev      Min      25%    Median       75%       Max  Variance
                  Citation Rate   235.0 11.977786 6.831064 1.000000 7.721340 11.155258 14.807399  68.50000 46.663431
                    Average FCR    23.0  4.629337 1.211145 0.259583 4.248517  4.748742  5.342243   6.33453  1.466873
            Research Efficiency  3645.0  0.918956 4.368187 0.000000 0.019691  0.109647  0.500000 190.20000 19.081057
Impact-Weighted Research Output 68166.0  0.006644 0.010288 0.000000 0.002791  0.004823  0.008005   1.00000  0.000106


100%|██████████| 1/1 [00:00<00:00, 874.36it/s]


# Impact Weighted Research Output, Benchmarking

The map shows the distribution of the benchmark with dark areas having higher weighted FCR values. The table shows the global metrics and how the top five countries compare


In [ ]:
# {echo=FALSE}
from google.cloud import bigquery
import pandas as pd
import plotly.express as px
import numpy as np

# Initialize the BigQuery client
client = bigquery.Client()

# 1. Calculate Global Benchmark Metric (No Country Grouping)
query_global = """
SELECT
    category_for.first_level.full[SAFE_OFFSET(0)].name AS field_name,
    metrics.field_citation_ratio AS fcr,
    COUNT(DISTINCT id) AS publication_count
FROM
    `covid-19-dimensions-ai.data.publications`
WHERE metrics.field_citation_ratio IS NOT NULL
GROUP BY field_name, fcr
"""

# Run the query and store the results in a Pandas DataFrame
df_global = client.query(query_global).to_dataframe()

# Calculate field-level average FCR
field_avg_fcr = df_global.groupby('field_name')['fcr'].transform('mean')

# Normalize FCR within fields
df_global['normalized_fcr'] = df_global['fcr'] / field_avg_fcr

# Calculate log-transformed publication count
df_global['log_publication_count'] = np.log1p(df_global['publication_count'])

# Calculate weighted FCR
df_global['weighted_fcr'] = df_global['normalized_fcr'] * df_global['log_publication_count']

# Get the max weighted FCR value
max_weighted_fcr = df_global['weighted_fcr'].max()

# Normalize the weighted fcr values.
df_global['normalized_weighted_fcr'] = df_global['weighted_fcr'] / max_weighted_fcr

# 2. Calculate Country-Level Aggregations (With Country Grouping)
query_country = """
SELECT
    research_org_country_names[SAFE_OFFSET(0)] AS country,
    category_for.first_level.full[SAFE_OFFSET(0)].name AS field_name,
    AVG(metrics.field_citation_ratio) AS avg_fcr,
    COUNT(DISTINCT id) AS publication_count
FROM
    `covid-19-dimensions-ai.data.publications`
WHERE metrics.field_citation_ratio IS NOT NULL
GROUP BY country, field_name
"""

# Run the query and store the results in a Pandas DataFrame
df_country = client.query(query_country).to_dataframe()

# Calculate field-level average FCR
field_avg_fcr_country = df_country.groupby('field_name')['avg_fcr'].transform('mean')

# Normalize FCR within fields
df_country['normalized_fcr'] = df_country['avg_fcr'] / field_avg_fcr_country

# Calculate log-transformed publication count
df_country['log_publication_count'] = np.log1p(df_country['publication_count'])

# Calculate weighted FCR
df_country['weighted_fcr'] = df_country['normalized_fcr'] * df_country['log_publication_count']

# Get the max weighted FCR value
max_weighted_fcr_country = df_country['weighted_fcr'].max()

# Normalize the weighted fcr values.
df_country['normalized_weighted_fcr'] = df_country['weighted_fcr'] / max_weighted_fcr_country

# Aggregate by country
country_df = df_country.groupby('country').agg(
    normalized_weighted_fcr=('normalized_weighted_fcr', 'mean'),
    publication_count=('publication_count', 'sum'),
    avg_fcr=('avg_fcr', 'mean')
).reset_index()

# Create choropleth map using plotly express
fig = px.choropleth(
    country_df,
    locations="country",
    locationmode="country names",
    color="normalized_weighted_fcr",
    hover_name="country",
    hover_data={
        "publication_count": ":.0f",  # Integer formatting
        "avg_fcr": ":.2f",  # Float formatting with 2 decimal places
    },
    color_continuous_scale="Blues",
    title="Impact-Weighted Research Output by Country"
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='patterson',  # Use Patterson projection
        lataxis = dict(range=[-60, 90]), #set range to exclude antartica
        lonaxis = dict(range=[-180,180]) #set range to exclude antartica
    )
)

fig.show()

def benchmark_top(df):
    """Benchmarks the top five countries based on normalized_weighted_fcr."""
    top_five = df.nlargest(6, 'normalized_weighted_fcr')
    print("Top Five Countries (Impact-Weighted Research Output):")
    for index, row in top_five.iterrows():
        print(f"{row['country']}: {row['normalized_weighted_fcr']:.4f}, Pub Count: {row['publication_count']}, Avg FCR: {row['avg_fcr']:.4f}")

benchmark_top(country_df)

# saved to gbq table for export
import pandas_gbq
pandas_gbq.to_gbq(country_df, "final_data.second_table_benchmark", project_id="covid-19-task")

100%|██████████| 1/1 [00:00<00:00, 8272.79it/s]


Top Five Countries (Impact-Weighted Research Output):
China: 0.3584, Pub Count: 155934, Avg FCR: 7.2680
United Kingdom: 0.3299, Pub Count: 103196, Avg FCR: 7.1782
United States: 0.3296, Pub Count: 327428, Avg FCR: 6.3678
Nauru: 0.3146, Pub Count: 3, Avg FCR: 57.7125
Australia: 0.3063, Pub Count: 43673, Avg FCR: 7.3793
Netherlands: 0.2815, Pub Count: 20136, Avg FCR: 7.6850


## Why is Nauru so high?
Look at the publication count. There are three pubs and one has a high value so the numbers are distorted. That pub has a single-author but Dimensions has taken all countries listed and weighted the FCR by each. This is only an issue when an an author has more than one affiliation. An odd instance but enough to point to a weakness in the data and the method chosen.

In [ ]:
# {echo=FALSE}
from google.cloud import bigquery
import pandas as pd
import numpy as np

# Initialize the BigQuery client
client = bigquery.Client()

# SQL query to get publications from Nauru
query_nauru = """
SELECT
    id,
    title,
    category_for.first_level.full[SAFE_OFFSET(0)].name AS field_name,
    metrics.field_citation_ratio AS fcr,
    COUNT(DISTINCT id) AS publication_count
FROM
    `covid-19-dimensions-ai.data.publications`
WHERE metrics.field_citation_ratio IS NOT NULL
AND ARRAY_LENGTH(research_org_country_names) > 0
AND research_org_country_names[SAFE_OFFSET(0)] = 'Nauru'
GROUP BY id, title, field_name, fcr
"""

# Run the query and store the results in a Pandas DataFrame
df_nauru = client.query(query_nauru).to_dataframe()

# Display the Nauru publications
if not df_nauru.empty:
    print("Publications from Nauru:")
    print(df_nauru.to_string(index=False))
else:
    print("No publications found for Nauru.")


100%|██████████| 1/1 [00:00<00:00, 6732.43it/s]

Publications from Nauru:
            id                                                                                          title            field_name    fcr  publication_count
pub.1154254235    {'preferred': 'The role of trade in pacific food security and nutrition', 'original': None}         Human Society  18.79                  1
pub.1143197480                  {'preferred': 'Medical Negligence Law in Papua New Guinea', 'original': None} Law and Legal Studies   0.00                  1
pub.1129143553 {'preferred': 'COVID-19: Fear, quackery, false representations and the law', 'original': None} Law and Legal Studies 193.27                  1


This plot visualizes the above table.

In [ ]:
# {echo=FALSE}

from google.cloud import bigquery
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Initialize the BigQuery client
client = bigquery.Client()

# SQL query to get data for normalized weighted FCR (using primary affiliation)
query_country = """
SELECT
    research_org_country_names[SAFE_OFFSET(0)] AS country,
    category_for.first_level.full[SAFE_OFFSET(0)].name AS field_name,
    AVG(metrics.field_citation_ratio) AS avg_fcr,
    COUNT(DISTINCT id) AS publication_count
FROM
    `covid-19-dimensions-ai.data.publications`
WHERE metrics.field_citation_ratio IS NOT NULL
GROUP BY country, field_name
"""

# Run the query and store the results in a Pandas DataFrame
df_country = client.query(query_country).to_dataframe()

# Calculate field-level average FCR
field_avg_fcr_country = df_country.groupby('field_name')['avg_fcr'].transform('mean')

# Normalize FCR within fields
df_country['normalized_fcr'] = df_country['avg_fcr'] / field_avg_fcr_country

# Calculate log-transformed publication count
df_country['log_publication_count'] = np.log1p(df_country['publication_count'])

# Calculate weighted FCR
df_country['weighted_fcr'] = df_country['normalized_fcr'] * df_country['log_publication_count']

# Get the max weighted FCR value
max_weighted_fcr_country = df_country['weighted_fcr'].max()

# Normalize the weighted fcr values
df_country['normalized_weighted_fcr'] = df_country['weighted_fcr'] / max_weighted_fcr_country

# Aggregate by country, including standard deviation
country_df = df_country.groupby('country').agg(
    normalized_weighted_fcr=('normalized_weighted_fcr', 'mean'),
    publication_count=('publication_count', 'sum'),
    avg_fcr=('avg_fcr', 'mean'),
    std_dev=('normalized_weighted_fcr', 'std')  # Calculate standard deviation
).reset_index()

# Calculate global mean normalized weighted FCR
global_mean_nwfcr = country_df['normalized_weighted_fcr'].mean()

# Get the top five countries (excluding Nauru)
top_five = country_df[country_df['country'] != 'Nauru'].nlargest(5, 'normalized_weighted_fcr')

# Create the bar plot for the top five countries with standard deviation error bars
fig = go.Figure()

# Add normalized weighted FCR bars with error bars
fig.add_trace(go.Bar(
    x=top_five['country'],
    y=top_five['normalized_weighted_fcr'],
    name='Impact-Weighted Research Output',
    marker_color='rgba(34, 139, 34, 0.2)',
    error_y=dict(type='data', array=top_five['std_dev'], visible=True),
))

# Add global mean line
fig.add_trace(go.Scatter(
    x=top_five['country'],
    y=[global_mean_nwfcr] * 5,
    mode='lines',
    name='Global Mean NWFCR',
    line=dict(color='darkgreen', width=2, dash='dash')
))

# Add annotations for text labels inside the bars
annotations = []
for i, row in top_five.iterrows():
    annotations.append(dict(
        x=row['country'],
        y=row['normalized_weighted_fcr'] / 2,  # Position in the middle of the bar
        text=f'{row["normalized_weighted_fcr"]:.2f}',
        showarrow=False,
        xanchor='center', #horizontal Alignment
        yanchor = 'middle', #vertical Alignment
        font=dict(color='black')
    ))

# Update layout
fig.update_layout(
    title='Top 5 Countries by Impact-Weighted Research Output (with Std. Dev.)',
    xaxis_title='Country',
    yaxis_title='Normalized Weighted FCR',
    template="plotly_white",
    annotations=annotations
)

fig.show()

# Network Map
This a nice interactive map.

In [ ]:
# {echo=FALSE}
from google.cloud import bigquery
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import numpy as np

# Initialize BigQuery client
client = bigquery.Client()

# SQL query to retrieve data and join with geo_data_coords
query = """
SELECT
    p.first_country_name,
    p.all_countries,
    p.field_name,
    g.COUNTRY as country,
    g.latitude,
    g.longitude
FROM
    `covid-19-task.fcr_data.fcr_percentile_subset` p
JOIN
    `covid-19-task.fcr_data.geo_data_coords` g ON p.first_country_name = g.country
"""

# Run query and store results in a DataFrame
df = client.query(query).to_dataframe()

# Explode and filter all_countries
df_exploded = df.explode('all_countries')
df_exploded = df_exploded.dropna(subset=['all_countries'])
df_exploded = df_exploded[df_exploded['first_country_name'] != df_exploded['all_countries']]

# Calculate field network sizes
field_network_sizes = df_exploded.groupby('field_name')['first_country_name'].nunique()

# Filter out fields with fewer than 5 unique countries
filtered_fields = field_network_sizes[field_network_sizes >= 5].index.tolist()
filtered_df = df_exploded[df_exploded['field_name'].isin(filtered_fields)]

#join to geo_data_coords
filtered_df = pd.merge(filtered_df, df[['first_country_name','latitude', 'longitude']].drop_duplicates(), on = 'first_country_name', how = 'left')

# Calculate co-occurrence matrix for filtered data
countries = filtered_df['first_country_name'].unique()
co_occurrence = pd.DataFrame(0, index=countries, columns=countries)

for index, row in filtered_df.iterrows():
    country1 = row['first_country_name']
    country2 = row['all_countries']
    if country1 in countries and country2 in countries:
        co_occurrence.loc[country1, country2] += 1
        co_occurrence.loc[country2, country1] += 1

# Create NetworkX graph
G = nx.from_pandas_adjacency(co_occurrence)

# Calculate network metrics
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)

# Filter out nodes with degree below 5
filtered_nodes = [node for node, deg in G.degree() if deg >= 5]
filtered_G = G.subgraph(filtered_nodes)

# Recalculate network metrics for filtered graph
degree_centrality = nx.degree_centrality(filtered_G)
betweenness_centrality = nx.betweenness_centrality(filtered_G)
closeness_centrality = nx.closeness_centrality(filtered_G)

# Identify top 10 influential countries
top_countries = sorted(degree_centrality, key=degree_centrality.get, reverse=True)[:10]

# Calculate node sizes (logarithmic scaling)
node_sizes = [np.log(degree_centrality.get(node, 1) + 1) * 20 if node in top_countries else 5 for node in filtered_G.nodes()] # Scale by 20, default size 5

# Get node positions (lat/lon)
node_positions = {}
for country in filtered_G.nodes():
    lat = df[df['first_country_name'] == country]['latitude'].iloc[0]
    lon = df[df['first_country_name'] == country]['longitude'].iloc[0]
    node_positions[country] = (lon, lat)  # NetworkX uses (lon, lat)

# Create node trace
node_lons = [node_positions[node][0] for node in filtered_G.nodes()]
node_lats = [node_positions[node][1] for node in filtered_G.nodes()]
node_fields = [df[df['first_country_name'] == node]['field_name'].iloc[0] for node in filtered_G.nodes()]

# Colorblind-friendly color list
color_list = ['#1b9e77', '#d95f02', '#7570b3', '#e7298a', '#66a61e', '#e6ab02', '#a6761d', '#666666']

# Map field names to colors
field_colors = {field: color_list[i % len(color_list)] for i, field in enumerate(set(node_fields))}

node_colors = [field_colors[field] for field in node_fields]

node_trace = go.Scattergeo(
    lon=node_lons,
    lat=node_lats,
    mode='markers',
    marker=dict(size=node_sizes, color=node_colors, opacity=0.7),
    hoverinfo='text',
    text=[f'{node}<br>Degree: {degree_centrality[node]:.2f}<br>Betweenness: {betweenness_centrality[node]:.2f}<br>Closeness: {closeness_centrality[node]:.2f}' for node in filtered_G.nodes()]
)

# Create edge traces with colors based on nodes
edge_traces = []
for edge in filtered_G.edges():
    source = node_positions[edge[0]]
    target = node_positions[edge[1]]
    color0 = field_colors[df[df['first_country_name'] == edge[0]]['field_name'].iloc[0]]
    edge_traces.append(go.Scattergeo(
        lon=[source[0], target[0]],
        lat=[source[1], target[1]],
        line=dict(width=filtered_G[edge[0]][edge[1]]['weight'] / 400, color=color0),  # Color based on node 0
        hoverinfo='none',
        mode='lines'
    ))

# Create layout
layout = go.Layout(
    title='Country Collaboration Network (Co-occurrence)',
    showlegend=False,
    geo=dict(
        scope='world',
        projection=dict(type='equirectangular'),
        showland=True,
        landcolor='rgb(217, 217, 217)',
        subunitcolor='rgb(217, 217, 217)',
        countrycolor='rgb(217, 217, 217)',
        showlakes=True,
        lakecolor='rgb(255, 255, 255)',
        showrivers=True,
        rivercolor='rgb(255, 255, 255)'
    ),
    width=1200,  # Increased width
    height=900,  # Increased height
)

# Create figure
fig = go.Figure(data=edge_traces + [node_trace], layout=layout)

# Show figure
fig.show()

# Network Metrics Table and Barplot

Here is the method

In [ ]:
# {echo=FALSE}

from google.cloud import bigquery
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots

# Initialize BigQuery client
client = bigquery.Client()

# SQL query to retrieve data and join with geo_data_coords
query = """
SELECT
    p.first_country_name,
    p.all_countries,
    p.field_name,
    g.COUNTRY as country,
    g.latitude,
    g.longitude
FROM
    `covid-19-task.fcr_data.fcr_percentile_subset` p
JOIN
    `covid-19-task.fcr_data.geo_data_coords` g ON p.first_country_name = g.country
"""

# Run query and store results in a DataFrame
df = client.query(query).to_dataframe()

# Explode and filter all_countries
df_exploded = df.explode('all_countries')
df_exploded = df_exploded.dropna(subset=['all_countries'])
df_exploded = df_exploded[df_exploded['first_country_name'] != df_exploded['all_countries']]

# Calculate field network sizes
field_network_sizes = df_exploded.groupby('field_name')['first_country_name'].nunique()

# Filter out fields with fewer than 5 unique countries
filtered_fields = field_network_sizes[field_network_sizes >= 5].index.tolist()
filtered_df = df_exploded[df_exploded['field_name'].isin(filtered_fields)]

# Calculate co-occurrence matrix for filtered data
countries = filtered_df['first_country_name'].unique()
co_occurrence = pd.DataFrame(0, index=countries, columns=countries)

for index, row in filtered_df.iterrows():
    country1 = row['first_country_name']
    country2 = row['all_countries']
    if country1 in countries and country2 in countries:
        co_occurrence.loc[country1, country2] += 1
        co_occurrence.loc[country2, country1] += 1

# Create NetworkX graph
G = nx.from_pandas_adjacency(co_occurrence)

# Calculate network metrics
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)

# Filter out nodes with degree below 5
filtered_nodes = [node for node, deg in G.degree() if deg >= 5]
filtered_G = G.subgraph(filtered_nodes)

# Recalculate network metrics for filtered graph
degree_centrality_filtered = nx.degree_centrality(filtered_G)
betweenness_centrality_filtered = nx.betweenness_centrality(filtered_G)
closeness_centrality_filtered = nx.closeness_centrality(filtered_G)

# Identify top 5 influential countries
top_countries = sorted(degree_centrality_filtered, key=degree_centrality_filtered.get, reverse=True)[:5]

# Create a DataFrame to store the results
results_df = pd.DataFrame(columns=['Country', 'Degree', 'Betweenness', 'Closeness'])

# Add descriptive stats for the entire dataset
all_data = {
    'Country': ['All (Mean)', 'All (Median)', 'All (Std Dev)', 'All (Min)', 'All (Max)'],
    'Degree': [np.mean(list(degree_centrality.values())), np.median(list(degree_centrality.values())), np.std(list(degree_centrality.values())), min(list(degree_centrality.values())), max(list(degree_centrality.values()))],
    'Betweenness': [np.mean(list(betweenness_centrality.values())), np.median(list(betweenness_centrality.values())), np.std(list(betweenness_centrality.values())), min(list(betweenness_centrality.values())), max(list(betweenness_centrality.values()))],
    'Closeness': [np.mean(list(closeness_centrality.values())), np.median(list(closeness_centrality.values())), np.std(list(closeness_centrality.values())), min(list(closeness_centrality.values())), max(list(closeness_centrality.values()))]
}

results_df = pd.concat([results_df, pd.DataFrame(all_data)], ignore_index=True)

# Add top 5 countries to the DataFrame
for country in top_countries:
    country_data = {
        'Country': country,
        'Degree': degree_centrality_filtered[country],
        'Betweenness': betweenness_centrality_filtered[country],
        'Closeness': closeness_centrality_filtered[country]
    }
    results_df = pd.concat([results_df, pd.DataFrame([country_data])], ignore_index=True)

# Print the results DataFrame
print(results_df)

# Calculate mean values for the entire dataset
mean_degree = np.mean(list(degree_centrality.values()))
mean_betweenness = np.mean(list(betweenness_centrality.values()))
mean_closeness = np.mean(list(closeness_centrality.values()))

# Create a DataFrame for the bar plot
plot_df = pd.DataFrame({
    'Country': ['Mean (All)'] + top_countries,
    'Degree': [mean_degree] + [degree_centrality_filtered[country] for country in top_countries],
    'Betweenness': [mean_betweenness] + [betweenness_centrality_filtered[country] for country in top_countries],
    'Closeness': [mean_closeness] + [closeness_centrality_filtered[country] for country in top_countries]
})


# save to gbq table for export
import pandas_gbq
pandas_gbq.to_gbq(plot_df, "final_data.network_plot", project_id="covid-19-task")

# Create subplots
fig = make_subplots(rows=1, cols=3, subplot_titles=('Degree', 'Betweenness', 'Closeness'))

# Degree plot
fig.add_trace(go.Bar(
    x=plot_df['Country'],
    y=plot_df['Degree'],
    name='Degree',
    marker_color='rgba(34, 139, 34, 0.4)',
    text=[f'{val:.2f}' for val in plot_df['Degree']],  # Text labels
    textposition='auto'  # Position the text automatically
), row=1, col=1)

# Betweenness plot
fig.add_trace(go.Bar(
    x=plot_df['Country'],
    y=plot_df['Betweenness'],
    name='Betweenness',
    marker_color='rgba(102, 0, 201, 0.4)',
    text=[f'{val:.2f}' for val in plot_df['Betweenness']],  # Text labels
    textposition='auto'  # Position the text automatically
), row=1, col=2)

# Closeness plot
fig.add_trace(go.Bar(
    x=plot_df['Country'],
    y=plot_df['Closeness'],
    name='Closeness',
    marker_color='rgba(255, 128, 0, 0.4)',
    text=[f'{val:.2f}' for val in plot_df['Closeness']],  # Text labels
    textposition='auto'  # Position the text automatically
), row=1, col=3)

# Update layout
fig.update_layout(
    title='Centrality Measures for Top Countries',
    xaxis_title='Country',
    yaxis_title='Centrality Measure Value',
    showlegend=False,
    template="plotly_white"
)

fig.show()

<ipython-input-28-bc6b58b11527>:85: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



          Country    Degree  Betweenness  Closeness
0      All (Mean)  0.282196     0.006340   0.588257
1    All (Median)  0.230769     0.000614   0.562500
2   All (Std Dev)  0.215943     0.017611   0.089579
3       All (Min)  0.008547     0.000000   0.414894
4       All (Max)  0.940171     0.151229   0.943548
5   United States  0.980769     0.092422   0.981132
6  United Kingdom  0.942308     0.073826   0.945455
7           China  0.798077     0.032875   0.832000
8           India  0.788462     0.033654   0.825397
9          Canada  0.759615     0.028011   0.806202


100%|██████████| 1/1 [00:00<00:00, 4554.08it/s]


# Bubble Plot showing Impactful Fields
Here is a nice plot that should probably have a table and needs explanation.

In [ ]:
# {echo=FALSE}

from google.cloud import bigquery
import pandas as pd
import plotly.express as px
import numpy as np

# Initialize BigQuery client
client = bigquery.Client()

# SQL query to retrieve data
query = """
SELECT
    category_for.first_level.full[SAFE_OFFSET(0)].name AS field_name,
    AVG(metrics.field_citation_ratio) AS avg_fcr,
    COUNT(DISTINCT id) AS publication_count
FROM
    `covid-19-dimensions-ai.data.publications`
WHERE metrics.field_citation_ratio IS NOT NULL
GROUP BY field_name
"""

# Run the query and store the results in a Pandas DataFrame
df = client.query(query).to_dataframe()

# Calculate field-level average FCR
field_avg_fcr = df.groupby('field_name')['avg_fcr'].transform('mean')

# Normalize FCR within fields
df['normalized_fcr'] = df['avg_fcr'] / field_avg_fcr

# Calculate log-transformed publication count
df['log_publication_count'] = np.log1p(df['publication_count'])

# Calculate weighted FCR
df['weighted_fcr'] = df['normalized_fcr'] * df['log_publication_count']

# Get the max weighted FCR value
max_weighted_fcr = df['weighted_fcr'].max()

# Normalize the weighted fcr values.
df['normalized_weighted_fcr'] = df['weighted_fcr'] / max_weighted_fcr

fig = px.scatter(
    df,
    x="normalized_weighted_fcr",
    y="publication_count",
    size="normalized_weighted_fcr",
    color="field_name",
    hover_name="field_name",
    hover_data=["publication_count", "avg_fcr"],
    title="Impact-Weighted Research Output by Field",
    labels={"normalized_weighted_fcr": "Normalized Weighted FCR", "field_name": "Field of Research"},
)

# Apply ggplot2 theme and viridis colorscale.
fig.update_layout(
    template="plotly_white",
    coloraxis_colorbar=dict(title="Field of Research"),
    coloraxis_colorscale="Viridis",
)

fig.show()